<a href="https://colab.research.google.com/github/Akshatt/Abstract-generation-using-text-summarizer/blob/master/Final_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
!pip install -q kaggle

In [0]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!ls

kaggle.json  sample_data


In [0]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [0]:
!cp kaggle.json ../root/kaggle

In [0]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

100% 1.92G/1.92G [00:22<00:00, 144MB/s]
100% 1.92G/1.92G [00:22<00:00, 92.2MB/s]


In [0]:
!kaggle datasets download -d yutanakamura/glove42b300dtxt

 99% 1.78G/1.80G [00:16<00:00, 110MB/s] 
100% 1.80G/1.80G [00:16<00:00, 117MB/s]


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import glob
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from  collections import OrderedDict
import os
import re
import string
import networkx as nx
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: document_parses/pmc_json/PMC7151927.xml.json  
  inflating: document_parses/pmc_json/PMC7151928.xml.json  
  inflating: document_parses/pmc_json/PMC7151931.xml.json  
  inflating: document_parses/pmc_json/PMC7151932.xml.json  
  inflating: document_parses/pmc_json/PMC7151933.xml.json  
  inflating: document_parses/pmc_json/PMC7151934.xml.json  
  inflating: document_parses/pmc_json/PMC7151935.xml.json  
  inflating: document_parses/pmc_json/PMC7151936.xml.json  
  inflating: document_parses/pmc_json/PMC7151937.xml.json  
  inflating: document_parses/pmc_json/PMC7151938.xml.json  
  inflating: document_parses/pmc_json/PMC7151939.xml.json  
  inflating: document_parses/pmc_json/PMC7151940.xml.json  
  inflating: document_parses/pmc_json/PMC7151941.xml.json  
  inflating: document_parses/pmc_json/PMC7151942.xml.json  
  inflating: document_parses/pmc_json/PMC7151943.xml.json  
  inflating: document_parses/pmc_json/PMC7151944.

In [0]:
meta=pd.read_csv("/content/metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
meta.shape

(63571, 18)

In [0]:
#meta=meta[((meta['pdf_json_files']==True) |(meta['pmc_json_files']==True))]
meta_sm=meta[['cord_uid','sha','pmcid','title','abstract','publish_time','url']]
meta_sm.drop_duplicates(subset ="title", keep = False, inplace = True)
meta_sm.loc[meta_sm.publish_time=='2020-12-31'] = "2020-03-31"
meta_sm.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

(61276, 7)

In [0]:
sys.path.insert(0, "../")
root_path = '/content/'

In [0]:
df = {"paper_id": [], "text_body": []}
df = pd.DataFrame.from_dict(df)
print(df)

collect_json = glob.glob(f'{root_path}/**/*.json', recursive=True)

for i,file_name in enumerate (collect_json):
    row = {"paper_id": None, "text_body": None}
    if i%2000==0:
        print ("====processed " + str(i)+ ' json files=====')
        print()

    with open(file_name) as json_data:
        data = json.load(json_data,object_pairs_hook=OrderedDict)
        if(file_name == '/content/kaggle.json'):
          continue
        if(file_name == '/content/sample_data/anscombe.json'):
          continue
        row['paper_id']=data['paper_id']
        
        body_list = []
       
        for _ in range(len(data['body_text'])):
            try:
                body_list.append(data['body_text'][_]['text'])
            except:
                pass

        body = "\n ".join(body_list)
        
        row['text_body']=body 
        df = df.append(row, ignore_index=True)

Empty DataFrame
Columns: [paper_id, text_body]
Index: []
====processed 0 json files=====

====processed 2000 json files=====

====processed 4000 json files=====

====processed 6000 json files=====

====processed 8000 json files=====

====processed 10000 json files=====

====processed 12000 json files=====

====processed 14000 json files=====

====processed 16000 json files=====

====processed 18000 json files=====

====processed 20000 json files=====

====processed 22000 json files=====

====processed 24000 json files=====

====processed 26000 json files=====

====processed 28000 json files=====

====processed 30000 json files=====

====processed 32000 json files=====

====processed 34000 json files=====

====processed 36000 json files=====

====processed 38000 json files=====

====processed 40000 json files=====

====processed 42000 json files=====

====processed 44000 json files=====

====processed 46000 json files=====

====processed 48000 json files=====

====processed 50000 json f

In [0]:
df.shape

(85371, 2)

In [0]:
#merge metadata df with parsed json file based on sha_id
merge1=pd.merge(meta_sm, df, left_on='sha', right_on=['paper_id'])
merge1.head()
merge1.shape

(45095, 9)

In [0]:
#merge metadata set with parsed json file based on pcmid
merge2=pd.merge(meta_sm, df, left_on='pmcid', right_on=['paper_id'])
merge2.head()
merge2.shape

(32738, 9)

In [0]:
#combine merged sha_id and pcmid dataset, remove the duplicate values based on file name
merge_final= merge2.append(merge1, ignore_index=True)
merge_final.drop_duplicates(subset ="title", keep = False, inplace = True)

In [0]:
#remove articles that are not related to COVID-19 based on publish time
corona=merge_final[(merge_final['publish_time']>'2019-11-01') & (merge_final['text_body'].str.contains('nCoV|Cov|COVID|covid|SARS-CoV-2|sars-cov-2'))]
corona.shape
#corona.head()

(7166, 9)

In [0]:
import re 
def clean_dataset(text):
    text=re.sub(r'^https?:\/\/.*[\r\n]*', '',str(text), flags=re.MULTILINE) #remove hyperlink
    text=re.sub(r'\\b[A-Z a-z 0-9._ - ]*[@](.*?)[.]{1,3} \\b', '', text)#remove email
    text=re.sub(r'^a1111111111 a1111111111 a1111111111 a1111111111 a1111111111.*[\r\n]*',' ',text)
    text=re.sub(r'  +', ' ',text ) #remove extra space
    text=re.sub(r's/ ( *)/\1/g','',text) 
    return text

In [0]:
import warnings
warnings.filterwarnings('ignore')
corona['text_body'] =corona['text_body'].apply(clean_dataset)
corona['title'] =corona['title'].apply(clean_dataset)
corona['abstract'] =corona['abstract'].apply(clean_dataset)
corona['text_body'] = corona['text_body'].map(lambda x: x.lower())
coro=corona.reset_index(drop=True)
coro.head()
coro.shape

(7166, 9)

In [0]:
coro['count_abstract'] = coro['abstract'].str.split().map(len)
#coro['count_abstract'].sort_values(ascending=True)

In [0]:
coro['count_text'] = coro['text_body'].str.split().map(len)
#coro['count_text'].sort_values(ascending=True)

In [0]:
coro2=coro[((coro['count_text']>500)&(coro['count_text']<8000))]
coro2.shape

(6265, 11)

In [0]:
coro2.to_csv("corona.csv",index=False)

In [0]:
test=coro2[coro2['count_abstract']<5]
test.head()
test.shape

(1556, 11)

In [0]:
test.to_csv("papers_with_no_abstract.csv",index=False)

In [0]:
gloveFile = "/content/glove.42B.300d.txt"

In [0]:
dim=300

In [0]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [0]:
def rem_ascii(s):
    return "".join()

In [0]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    processed = re.sub(r"\d+","",punc_free)
    return processed

In [0]:
def loadGloveModel(gloveFile):
    # Glove file looks like this
    # the 0.27204 -0.06203 -0.1884 0.023225 -0.018158 ....
    # and -0.18567 0.066008 -0.25209 -0.11725 0.26513 ....
    # to 0.31924 0.06316 -0.27858 0.2612 0.079248 .....
    # of 0.060216 0.21799 -0.04249 ....
    # .....
    # Word Coeff1 Coeff2 Coeff3 Coeff4 Coeff5 Coeff6.....
    word_embeddings = {}
    f = open(gloveFile, encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    # word_embeddings = {'the': [0.27204 -0.06203 -0.1884 0.023225 -0.018158 ....], 'and':[-0.18567 0.066008 ....], etc......}
    return word_embeddings

In [0]:
word_embeddings = loadGloveModel(gloveFile)

In [0]:
print("Vocab Size: ",len(word_embeddings))

Vocab Size:  1917494


In [0]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [0]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [0]:
def abstract_creation(text, k):
    text_bod = text['text_body']    
    sentences = split_into_sentences(text_bod)
   
    # Actually a for loop within a for loop. What it does is splits the list of sentences into individual sentences. It then splits
    # these indivdual sentences into words. Uses the Glove file to access the vectors of each word. Averages the 300 dimensional 
    # vectors of each word to get a 300 dimensional vector of a sentence. Similar sentences will have similar vectors?
    sentence_vectors = []
    for i in sentences:
        if len(i)!=0:
            # np.zeros((dim,)) -> gives an array of 300 Zeros -> [0. 0. 0. 0. 0. ............. 0. 0. 0. 0.]
            v = sum([word_embeddings.get(w, np.zeros((dim,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((dim,))
        sentence_vectors.append(v)

    # Now we have 300 dimensional vector for each sentence of the article.
    # Next step is to create similarity matrix that keeps cosine distance of each sentecne to every other sentence

    # MxN matrix where M, N is the number of sentences
    sim_mat = np.zeros([len(sentences),len(sentences)])
    # Calculating similarity between sentences
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i!=j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,dim),sentence_vectors[j].reshape(1,dim))[0,0]

    sim_mat = np.round(sim_mat,3)

    # Basically we'll be using the text rank algo
    # More impor. websites are likely to receive more links from other websites
    # Similary we'll see how one sentence is similar to other sentences

    nx_graph = nx.from_numpy_array(sim_mat)
    try:
        scores = nx.pagerank(nx_graph)
    except:
        message = "Page Rank Algo didn't converge"
        test.iloc[k, test.columns.get_loc('abstract')] = message
        return
    ranked_sentences = sorted(((scores[i],i)for i,s in enumerate(sentences)),reverse=True)
    # [(Score, Sentence number),(Score, Sentence number), ....  ,(Score, Sentence number)] in descenfing order of Score
    #[(0.08676223755889753, 7), (0.0854425960863795, 2), (0.08534797511096875, 11) ........ , (0.07483409026129421, 6)]

    max_abstr_len_in_words = 500
    new_ranked_sent = list()
    curr_abstr_len = 0
    for i in ranked_sentences:
        sent_length = len(sentences[i[1]].split())
        curr_abstr_len = curr_abstr_len + sent_length
        if(curr_abstr_len > max_abstr_len_in_words):
            break
        new_ranked_sent.append(i)

    arranged_sentences = sorted(new_ranked_sent , key=lambda x:x[1])

    # Abstract that is produced based on text body using TextRank Algo - Max 500 Words
    created_abstract = "\n".join([sentences[x[1]] for x in arranged_sentences])
    test.iloc[k, test.columns.get_loc('abstract')] = created_abstract

In [0]:
for k in range(0, len(test)):
    text = test.iloc[k]
    abstract_creation(text, k)
    if(k%100 == 0):
      print(k, "research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques")  

test.to_csv("papers_with_no_abstract_now_having_an_abstract_trying_again_HundredIter.csv",index=False)

0 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
100 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
200 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
300 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
400 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
500 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
600 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
700 research papers have been parsed and their subsequent abstracts have been made after using text summarization techniques
80

In [0]:
import csv
filename = "papers_with_no_abstract_now_having_an_abstract_trying_again_HundredIter.csv"
f = []
with open(filename, 'r', encoding = 'cp850') as csvfile:
    csvreader = csv.reader(csvfile)
    f = next(csvreader)  
    data = [[field] for field in f]
    for row in csvreader:
        for i,r in enumerate(row):
            data[i].append(r)

In [0]:
keyword = input("Enter the keywords present in the article title you are looking for:\n")
#storing all the keywords in a list
kwd = [k for k in keyword.split()]

Enter the keywords present in the article title you are looking for:
lockdown vaccine hydroxychloroquine


In [0]:
#creating a dictionary where keyword is the key and titles containing keyword are its value
dict = {}
for i in kwd:
    for j in range(len(data[3])-1):
        if i in data[3][j]:
            if i in dict:
                dict[i].append(data[3][j])
            else:
                dict[i] = [data[3][j]]                

In [0]:
print("\nFollowing articles found containing keywords mentioned:\n")
#printing all the titles found for the user to choose from and storing the title number in a new dictionary with title name as its value
dct2 = {}
iter = 1
for k in dict.keys():
    for v in dict[k]:
        print("{}. {}\n".format(iter,v))
        dct2[iter] = v
        iter += 1


Following articles found containing keywords mentioned:

1. ExpertsÔÇÖ request to the Spanish Government: move Spain towards complete lockdown

2. Why India needs to extend the nationwide lockdown

3. Sport in the tracks and fields of the corona virus: critical issues during the exit from lockdown

4. Addressing psychosocial problems associated with the COVID-19 lockdown

5. Ensuring global access to COVID-19 vaccines

6. Two Middle East respiratory syndrome vaccines: first step for other coronavirus vaccines?

7. Corona (COVID-19) time musings: Our involvement in COVID-19 pathogenesis, diagnosis, treatment and vaccine planning

8. COVID-19 and risks to the supply and quality of tests, drugs, and vaccines

9. No evidence of rapid antiviral clearance or clinical benefit with the combination of hydroxychloroquine and azithromycin in patients with severe COVID-19 infection

10. Does hydroxychloroquine combat COVID-19? A timeline of evidence

11. COVID-19 research has overall low methodol

In [0]:
num = 0
while True:     
    num = int(input("\nEnter the title number whose abstract you wish to see or -1 to exit:\n\n\n"))
    # look for the title name from from given title number
    # keep asking for the title number if entered number is invalid.
    if num == -1:
        break 
    count = 0
    while count != 1:   
        if num in dct2.keys():
            for j in range(len(data[3])-1):
                if dct2[num] in data[3][j]:
                    print("\n",data[4][j])
                    count = 1
        else:
            print("Title number not found! try again!")


Enter the title number whose abstract you wish to see or -1 to exit:


14

 a 66-year-old woman with a past medical history of rheumatoid arthritis, pulmonary fibrosis, and asthma, on chronic methotrexate and oral and inhaled steroids, presented on  the combination of azithromycin and chloroquine/ hydroxychloroquine has been shown in a small number of covid-19 patients to have effective virucidal potency.
the addition of lidocaine or mexiletine may allow this type of combination therapy even in patients who have a prolonged qt interval.
we developed and provided a flow chart to manage the risk of qt prolongation and tdp while treating these patients in the hospital with combination therapy.
owing to several similar consults in covid-19 patients with borderline or increased qtc intervals, we developed a flow chart that addresses qt interval management in order to help clinicians taking care of covid-19 patients who require chloroquine or hydroxychloroquine with or without azithromycin 